# **Selección de variables de interés**

Uso de técnicas para la selección de variables con mayor importancia, evitando supuestos de multicolinealidad.

In [1]:
# Librerias -----------------------------
# Manipulación de datos
import pandas as pd
import numpy as np
import time
# Visualización
import matplotlib.pyplot as plt

# sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report, 
    ConfusionMatrixDisplay, precision_recall_curve, roc_curve, auc
)
from sklearn.ensemble import RandomForestClassifier

# Importar funciones necesarias
from statsmodels.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
file = r"C:\Users\Hp\MACHINE\MINI_PRY_1\data\data_lend.csv"
df = pd.read_csv(file, low_memory=False)
print("Dimensiones del dataset:", df.shape)

# Listado de imputaciones
imputar_cero = ['monto_total_cobranzas', 'lineas_credito_12m', 'cuentas_tarjeta_credito',
                'cuentas_hipotecarias', 'quiebras_publicas', 'cobranzas_12m', 'castigos_12m'
]
imputar_mediana = ['meses_ultima_consulta', 'antiguedad_laboral','porcentaje_sin_moras',
                   'limite_credito_total', 'meses_tarjeta_nueva', 'uso_credito_revolvente',
                   'dti', 'saldo_revolvente', 'monto_mora'
]
for col in imputar_cero:
    if col in df.columns:
        df[col] = df[col].fillna(0)
for col in imputar_mediana:
    if col in df.columns:
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)
missing_summary = df[imputar_cero + imputar_mediana].isnull().sum()

Dimensiones del dataset: (1345310, 40)


* **Variables descartadas por correlación**

In [3]:
# Descarte de las variables que general alta correlación.
df.drop(columns=['cuota_mensual', 'categoria_credito'], inplace=True)

* Prueba VIF (Variance Inflation Factor)

Para detectar multicolinealidad entre variables independientes

In [4]:
# VIF

#Selecciona solo variables numéricas (excluyendo la variable objetivo 'default')
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
num_cols.remove('default')

X = add_constant(df[num_cols])

# Calcula VIF para cada variable
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data[vif_data['VIF'] > 2].sort_values(by='VIF', ascending=False))

Para las variables con VIF mayor a 2, `promedio_fico` es la única variable de la salida, aunque por su valor no indica problema de colinealidad.

* Modelo RandomForest

Selección de características importantes por FeaturesImportance

In [11]:
# Modelo Random Forest
X = df.drop('default', axis=1)
y = df['default']

num_cols = X.select_dtypes(include=['float64','int64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', StandardScaler(), num_cols)
    ],
    remainder='passthrough'
)
X_processed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, stratify=y, random_state=42
)

In [13]:
# Entrenar modelo 
rf = RandomForestClassifier(
    n_estimators=200, max_depth=10, class_weight='balanced',
    random_state=42, n_jobs=-1
)
rf.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
# Importancia de variables
importances = rf.feature_importances_
feature_names = preprocessor.get_feature_names_out()
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importances})
feat_imp = feat_imp.sort_values(by='importance', ascending=False)
print(feat_imp.head(20))

                        feature  importance
24       num__tuvo_acuerdo_pago    0.284473
29            num__tasa_interes    0.272178
44             num__plazo_meses    0.135213
42           num__promedio_fico    0.058111
27                     num__dti    0.035960
40    num__limite_credito_total    0.024973
48    num__estado_verif_ingreso    0.022574
50   num__anio_emision_prestamo    0.021907
28          num__monto_aprobado    0.019970
34      num__lineas_credito_12m    0.016464
21           num__ingreso_anual    0.015309
30    num__cuentas_hipotecarias    0.012981
1   cat__tipo_vivienda_MORTGAGE    0.012392
32   num__meses_ultima_consulta    0.009717
5       cat__tipo_vivienda_RENT    0.009366
38  num__uso_credito_revolvente    0.008480
31     num__meses_tarjeta_nueva    0.007023
51   num__anio_apertura_credito    0.005061
37        num__saldo_revolvente    0.004423
41   num__total_cuentas_credito    0.002748


* **Variables de interés**

A partir del anterior listado se seleccionan las siguientes variables para la generación de modelos:

In [19]:
importancie = [
    'monto_aprobado', 'tasa_interes', 'plazo_meses', 'motivo_prestamo', 

    'ingreso_anual', 'tipo_vivienda', 'antiguedad_laboral', 'estado_verif_ingreso',

    'promedio_fico', 'anio_apertura_credito', 'cuentas_hipotecarias', 'total_cuentas_credito',
    'cuentas_tarjeta_credito', 'saldo_revolvente', 'uso_credito_revolvente', 'limite_credito_total',

    'dti', 'meses_ultima_consulta', 'meses_tarjeta_nueva', 'lineas_credito_12m', 'porcentaje_sin_moras',
    
    'moras_2_year', 'monto_total_cobranzas', 'tuvo_acuerdo_pago', 
    'subcategoria_credito', 'anio_emision_prestamo', 'mes_emision_prestamo', 
    
    'default'
]

In [20]:
df_filtrado = df[importancie]
df_filtrado.to_csv('data_to_model.csv', index=False)

In [21]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345310 entries, 0 to 1345309
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   monto_aprobado           1345310 non-null  float64
 1   tasa_interes             1345310 non-null  float64
 2   plazo_meses              1345310 non-null  int64  
 3   motivo_prestamo          1345310 non-null  object 
 4   ingreso_anual            1345310 non-null  float64
 5   tipo_vivienda            1345310 non-null  object 
 6   antiguedad_laboral       1345310 non-null  float64
 7   estado_verif_ingreso     1345310 non-null  int64  
 8   promedio_fico            1345310 non-null  float64
 9   anio_apertura_credito    1345310 non-null  int64  
 10  cuentas_hipotecarias     1345310 non-null  float64
 11  total_cuentas_credito    1345310 non-null  float64
 12  cuentas_tarjeta_credito  1345310 non-null  float64
 13  saldo_revolvente         1345310 non-null 